BOARDCOVER2

[algospot의 BOARDCOVER2 링크](https://algospot.com/judge/problem/read/BOARDCOVER2)  
  
### 문제  
H×W 크기의 게임판과 한 가지 모양의 블록이 여러 개 있습니다. 게임판에 가능한 많은 블록을 올려놓고 싶은데, 게임판은 검은 칸과 흰 칸으로 구성된 격자 모양을 하고 있으며 이 중에서 흰 칸에만 블록을 올려놓을 수 있습니다. 이때 블록들은 자유롭게 회전해서 놓을 수 있지만, 서로 겹치거나, 격자에 어긋나게 덮거나, 검은 칸을 덮거나, 게임판 밖으로 나가서는 안 됩니다.  
  
![](http://algospot.com/media/judge-attachments/556e7cbd2262a90832a95e0106df50bd/boardcover2.png)  
  
위 그림은 예제 게임판과 L 자 모양의 블록으로 이 게임판을 덮는 방법을 보여줍니다. 게임판에는 15개의 흰 칸이 있고, 한 블록은 네 칸을 차지하기 때문에 그림과 같이 최대 세 개의 블록을 올려놓을 수 있지요. 게임판과 블록의 모양이 주어질 때 최대 몇 개의 블록을 올려놓을 수 있는지 판단하는 프로그램을 작성하세요.  
  
### 출력   
게임판에 놓을 수 있는 최대의 블록 수를 출력합니다.  

solution의  
H, W : 보드의 높이.너비  
R, C : 격자의 높이.너비  
board : 보드의 상태 나타내는 이중 리스트  
frag : 보드의 격자 나타내는 이중 리스트  
(1 <= 너비.높이 <= 10)  

In [1]:
from copy import deepcopy

def preprocessing(R, C, frag): # 전처리하는 모듈
    frags = set() # 격자의 상대 좌표 저장할 set 객체
    f1 = []
    f2 = []
    f3 = []
    f4 = []
    # 총 4가지의 격자 놓는 방법이 존재한다.
    y = -1
    x = -1
    # 기준 좌표 y,x
    num_block = 0 # 격자가 가지는 블록의 크기(수)
    for r in range(R): 
        for c in range(C):
            if frag[r][c] == '#':
                num_block += 1
                if y == - 1 and x == -1: # 제일 처음 발견된 격자의 부분을 기준으로 삼는다.
                    y = r
                    x = c
                f1.append((r-y, c-x)) # 0도
                f2.append((-(r-y), -(c-x))) # 180도
                f3.append((c-x, -(r-y))) # 90도
                f4.append((-(c-x), (r-y))) # 270도
                # 격자를 돌려놓은 각도에 따라 절대 위치 좌표를 저장

    frags.add(tuple(f1))
    frags.add(tuple(f2))
    frags.add(tuple(f3))
    frags.add(tuple(f4))
    # 절대 좌표를 저장
    frags = list(frags) # 리스트화 한다.
    return frags, num_block # 격자의 절대좌표, 격자가 가지는 블록의 개수
    
def calculate(max_val, H, W, board, fragments, total_sharp, num_block, init_y = 0, init_x = 0):
    # 최대 채울 수 있는 격자를 구하는 모듈
    # max_val : 현재까지의 최대 격자 수
    # total_sharp : 여태까지 이미 확인한 블록 수 + 확인하지 않은 블록 중에 #인 블록의 수
    # num_block : 격자를 구성하는 블록의 수
    # init_y, init_x : 현재 블록의 (y,x)위치까지는 모두 확인한 상태라는 뜻
    
    if init_y == H-1 and init_x == W-1: # 만약 모든 블록의 좌표를 확인했을 때
        return 0 # 0 반환
    if ((H*W - total_sharp) // num_block) < max_val: # pruning(가지치기)
        # 만약 현재 추가할 수 있는 최대 블록수 < 현재 최대값 이라면 가지치기 시행
        return 0 # 0 반환
    
    answer = 0 # 답을 저장할 변수
    
    for h in range(init_y, H): # 이미 지나친 행은 지나지 않는다.
        for w in range(W):
            if init_y == h and init_x >= w: # 만약 이미 본 부분이라면
                continue # 넘어간다.
            if board[h][w] == '.': # 현재 보는 보드의 블록이 비어있다면
                total_sharp += 1 # 확인한 블록 수 증가
                for f in fragments: # 격자들에 대해서
                    blocking = True 
                    new_board = deepcopy(board) # 보드 deepcopy
                    for y,x in f: 
                        if  h+y >= H or h+y < 0 or w+x >= W or w+x < 0: # 격자를 넣어서 보드를 벗어날 경우
                            blocking = False
                            break
                        if new_board[h+y][w+x] == '#': # 만약 격자가 보드 안에 들어가지 못하는 경우
                            blocking = False
                            break 
                        new_board[h+y][w+x] = '#' # 위의 2가지 경우가 아니라면 격자를 채움
                        
                    if blocking: # 만약 격자를 문제없이 보드위에 넣었다면
                        answer = max(answer, 1+calculate(answer, H, W, new_board, fragments, total_sharp+num_block-1, num_block ,h, w))
                        # 보드에 넣을 수 있는 최대 격자 수를 저장
                if answer != 0: # 현재 좌표에서 격자가 들어갔을 경우에
                    answer = max(answer, calculate(answer, H, W, board, fragments, total_sharp, num_block,h, w))
                    # 현재 좌표에서 격자를 안 넣었을 경우와 비교해서 최대값을 저장
                    return answer # 답을 반환
    return answer # 답을 반환
                    
                        
    
def solution(H, W, R, C, board, frag):
    answer = 0 # 답
    total_sharp = 0 # 현재 보드에서 #의 개수
    for i in range(H):
        for j in range(W):
            if board[i][j] == '#':
                total_sharp += 1
    fragments, num_block = preprocessing(R, C, frag) # 격자에 대해서 전처리
    answer = calculate(0, H, W, board, fragments, total_sharp, num_block)
    # 보드에 넣을 수 있는 최대 격자 수 반환
    return answer # 답 반환

In [2]:
H = 4
W = 7
R = 2
C = 3
board = [["#","#",".","#","#",".","."],
         ["#",".",".",".",".",".","."],
         ["#",".",".",".",".","#","#"],
         ["#",".",".","#","#","#","#"]]
frag = [["#","#","#"],["#",".","."]]

In [3]:
solution(H, W, R, C, board, frag)

3

In [4]:
H = 5
W = 10
R = 3
C = 3
board = [[".",".",".",".",".",".",".",".",".","."],
         [".",".",".",".",".",".",".",".",".","."],
         [".",".",".",".",".",".",".",".",".","."],
         [".",".",".",".",".",".",".",".",".","."],
         [".",".",".",".",".",".",".",".",".","."]]
frag = [[".","#","."],["#","#","#"],[".",".","#"]]

In [5]:
solution(H, W, R, C, board, frag)

8